***Empiesa la fiehta***

# Libraries

In [37]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.naive_bayes import MultinomialNB


#acordarse del orden de los imports modificarlo


After a while looking for interesting datasets we decided to choose a cybersecutity dataset of threat detection and reaction strategies. Where our target column is 'Response Action'(Wich response was performed by the IA).

We proceed to train and try a model to predict this target.

# Reading the dataset

In [38]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [39]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\desqu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
fake_news_dataset = pd.read_csv('fake_and_real_news.csv')

In [41]:
fake_news_dataset["Text"] = fake_news_dataset["Text"].str.lower()
fake_news_dataset.columns = fake_news_dataset.columns.str.lower()

In [42]:
fake_news_dataset.describe()

,text,label
count,9900,9900
unique,9865,2
top,highlights: the trump presidency on april 13 a...,Fake
freq,8,5000


In [68]:
duplicates = fake_news_dataset[fake_news_dataset.duplicated(subset=["text"], keep=False)]
non_unique_texts = duplicates["text"]
print(non_unique_texts)
print(non_unique_texts.value_counts())

251     highlights: the trump presidency on march 31 a...
784     highlights: the trump presidency on april 13 a...
811     highlights: the trump presidency on april 21 a...
929     turkey urges u.s. to review visa suspension as...
980     trump taps fed centrist powell to lead u.s. ce...
1073    trump interviews four for fed chair job, to de...
1186    trump asks supreme court to block travel ban r...
1197    trump jr. emails suggest he welcomed russian h...
1427    'nuclear option' fallout? more extreme u.s. ju...
1576    highlights: the trump presidency on april 13 a...
1638    republicans unveil tax cut bill, but the hard ...
1764    highlights: the trump presidency on april 26 a...
2010    former trump security aide was russia blackmai...
2015    trump urges nfl to ban players kneeling during...
2173    trump asks supreme court to block travel ban r...
2276    trump tries to rally support for healthcare bi...
2529    highlights: the trump presidency on april 28 a...
2887    trump 

In [62]:
non_unique_texts = duplicates["text"]
print(non_unique_texts)

array([' top trump surrogate brutally stabs him in the back: ‘he’s pathetic’ (video) it s looking as though republican presidential candidate donald trump is losing support even from within his own ranks. you know things are getting bad when even your top surrogates start turning against you, which is exactly what just happened on fox news when newt gingrich called trump  pathetic. gingrich knows that trump needs to keep his focus on hillary clinton if he even remotely wants to have a chance at defeating her. however, trump has hurt feelings because many republicans don t support his sexual assault against women have turned against him, including house speaker paul ryan (r-wi). so, that has made trump lash out as his own party.gingrich said on fox news: look, first of all, let me just say about trump, who i admire and i ve tried to help as much as i can. there s a big trump and a little trump. the little trump is frankly pathetic. i mean, he s mad over not getting a phone call? trump s

In [43]:
fake_news_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9900 entries, 0 to 9899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    9900 non-null   object
 1   label   9900 non-null   object
dtypes: object(2)
memory usage: 154.8+ KB


In [44]:
fake_news_dataset["label"].value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [45]:
fake_news_dataset

,text,label
0,top trump surrogate brutally stabs him in the...,Fake
1,u.s. conservative leader optimistic of common ...,Real
2,"trump proposes u.s. tax overhaul, stirs concer...",Real
3,court forces ohio to allow millions of illega...,Fake
4,democrats say trump agrees to work on immigrat...,Real
...,...,...
9895,wikileaks admits to screwing up immensely wit...,Fake
9896,trump consults republican senators on fed chie...,Real
9897,trump lawyers say judge lacks jurisdiction for...,Real
9898,watch: right-wing pastor falsely credits trum...,Fake


In [55]:
label_dict= {'Fake': 0, 'Real': 1,}

fake_news_dataset['label']= fake_news_dataset['label'].replace(label_dict).infer_objects(copy=False)

In [52]:
tokens = fake_news_dataset['text'].apply(nltk.word_tokenize)

lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

tokens = tokens.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

tokens = tokens.apply(lambda x: [word for word in x if word not in stop_words])

In [56]:
texts = tokens.apply(' '.join)

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(texts)

y = fake_news_dataset['label']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:


model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [61]:


predictions = model.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, predictions))

Accuracy:  0.9787878787878788
